# Training an AI model in Audio Detection

## Introduction

Previously, this same notebook was used to train an AI model from the <a href='https://www.kaggle.com/mehmetokuyar/urbansounds'>UrbanSounds dataset</a> that can be found on <a href='Kaggle.com'>Kaggle.com</a>. As a result, the classifer files for detecting the following: 
1. air conditioner
2. dog barking
3. car horns
4. children playing
5. drilling noises
6. idle engine noises
7. gun shots
8. jackhammers
9. sirens
10. street music

were generated and are used to classify with great accuracy whether any of those items is playing in an audio file. That's the model used to detect if an air conditioner is on in the `ACDetector.ipynb` notebook. These classifer files are found in the current directory and are labeled with the name `urbanSounds*`. There are of course limitations to this trained model. For example, similar sounds to an air conditioner (e.g., fans, white noise) can yield false positives because there's no way to distinguish them. So, the greater the number of different audio classes, the higher the accuracy in detection of the model.

The present notebook will be used as an example on how to train an AI model in heartbeat detection with only a few lines of code; that is done thanks to the powerful analysis tools provided by pyAudioAnalysis and the partially labeled dataset downloaded from [Heartbeat Sounds](https://www.kaggle.com/kinguistics/heartbeat-sounds). Take a look at the website of the dataset to see how it's organized. These audio files will not be included in this repository due to their large size (832 audio files--159.4MB), but the classifier files will be included in the `audio_training_dataset/heartbeat_sounds/classifier_files/` directory. Make sure to change the files' location in the sections below if you decide to use those files instead of generating your own. 

Following the steps below will produce an AI model able to detect heartbeat sounds from scratch:  

1. We will use the `pyAudioAnalysis` library to train an AI model to classify heartbeat sounds
2. Download the dataset from the website above
    1. Download the set_a and set_b zip files only
3. Extract the WAV files and move them to the `audio_training_dataset/heartbeat_sounds/` directory
4. Follow the sections below

## File Conversions & Organize Dataset

pyAudioAnalysis only accepts files with a 44100Hz sample rate to do its training and classification, so the following code section will go through every heartbeat audio file in `audio_training_dataset/heartbeat_sounds/` and translate them to 44100Hz at 705kbps bitrate if it's not already in that format. 

And in order to take full advantage of the labeled dataset of heartbeat sounds, the audio files can be seperated into different classes. Those classes are artifact, Aunlabelled, Bunlabelled, extrahls, extrastrole, murmur, murmur_noisymurmur, normal, normal_noisynormal. The following code will also organize the audio files in `audio_training_dataset/heartbeat_sounds/` into their respective folders within that same directory. 

Take a look at the terminal window where this jupyter notebook is running and the `audio_training_dataset/heartbeat_sounds/` directory to see the magic happen

In [5]:
import os
import wave
from tqdm import tqdm

In [7]:
directory = 'audio_training_dataset/heartbeat_sounds/'
folders = ['artifact__', 'Aunlabelledtest__', 'Bunlabelledtest__', 'extrahls__', 'extrastole__', 'murmur__', 
           'murmur_noisymurmur_', 'normal__', 'normal_noisynormal_']

for file_name in tqdm(os.listdir(directory)): # scans the directory for all files
    try:
        with wave.open(directory + file_name, "rb") as wav_file:
            sample_rate = wav_file.getframerate() # gets sample rate
            if (sample_rate != 44100):
                os.system(f"ffmpeg -i {directory + file_name} -ar 44100 {directory + file_name} -y") # converts noncompliant files to 44100Hz
            wav_file.close()
            
            for name in folders: 
                if name in file_name: # will check which folder to be put into
                    location = directory + name
                    part1 = f'DIR="{location}"\n' 
                    part2 = 'if [ -d "$DIR" ]; then \n' # checks if folder exists 
                    part3 = f'mv {directory + file_name} {location}\n' # moves file into folder 
                    part4 = f'else mkdir {location}; mv {directory + file_name} {location}\n' # creates folder and moves file
                    part5 = 'fi'
                    os.system(part1 + part2 + part3 + part4  + part5) # executing command
                    break

    except wave.Error:
        print(directory + file_name + " is not a WAV file")
        continue

 88%|████████▊ | 737/834 [02:43<00:25,  3.75it/s]

audio_training_dataset/heartbeat_sounds/readme.txt is not a WAV file


 93%|█████████▎| 772/834 [02:51<00:14,  4.33it/s]

audio_training_dataset/heartbeat_sounds/.gitignore is not a WAV file


100%|██████████| 834/834 [03:03<00:00,  4.56it/s]


## AI Training

The `pyAudioAnalysis` library provides us with a very convenient wrapper function, `extract_features_and_train()`, that extracts audio features for classifier training. This function takes five parameters:

1. **paths**: a list of paths of directories. Each directory contains a single audio class whose samples are stored in seperate WAV files.
2. **mid_window**: mid-term window length
3. **mid_step**: mid-term step length
4. **short_window**: short-term window length
5. **short_step**:  short-term step length
6. **classifier_type**: "svm" or "knn" or "randomforest" or "gradientboosting" or "extratrees"
7. **model_name**: name of the model to be saved
8. **computeBEAT**: boolean to run beat analysis (useful for music)

This function does not return anything, rather the resulting classifier along with the respective model parameters are saved on files, similar to those of the `urbanSounds` model. They are saved in the parent directory of the program that calls `extract_features_and_train()`; in this case, `AC_Detector/`.

For more information visist the website's [classification page](https://github.com/tyiannak/pyAudioAnalysis/wiki/4.-Classification-and-Regression). 

**WARNING:** this process may take a few hours. Usually up until the Param = 20.00000 tests finish.

We call the function as follows:

In [8]:
from pyAudioAnalysis import audioTrainTest as aT

In [12]:
aT.extract_features_and_train([directory + folders[0], directory + folders[1], directory + folders[2], 
                               directory + folders[3], directory + folders[4], directory + folders[5], 
                               directory + folders[6], directory + folders[7], directory + folders[8]], 
                              1.0, 1.0, aT.shortTermWindow, aT.shortTermStep, "svm", "heartbeats")

Analyzing file 1 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201012172012.wav
Analyzing file 2 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105040918.wav
Analyzing file 3 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105041959.wav
Analyzing file 4 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105051017.wav
Analyzing file 5 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105060108.wav
Analyzing file 6 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105061143.wav
Analyzing file 7 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105190800.wav
Analyzing file 8 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201105280851.wav
Analyzing file 9 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifact__201106010559.wav
Analyzing file 10 of 40: audio_training_dataset/heartbeat_sounds/artifact__/artifa

Analyzing file 36 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201106171155.wav
Analyzing file 37 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201106191034.wav
Analyzing file 38 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201106211725.wav
Analyzing file 39 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201106212102.wav
Analyzing file 40 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201108011111.wav
Analyzing file 41 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201108011113.wav
Analyzing file 42 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201108011116.wav
Analyzing file 43 of 52: audio_training_dataset/heartbeat_sounds/Aunlabelledtest__/Aunlabelledtest__201108011117.wav
Analyzing file 44 of 52: audio_training_dataset/heartbeat_sounds

Analyzing file 51 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__139_1306519274653_C.wav
Analyzing file 52 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__141_1306520154450_A.wav
Analyzing file 53 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__141_1306520154450_D.wav
Analyzing file 54 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__142_1306763049574_C.wav
Analyzing file 55 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__143_1306763822290_D.wav
Analyzing file 56 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__144_1306522408528_A.wav
Analyzing file 57 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__145_1307987561278_A.wav
Analyzing file 58 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__145_1307987561278_D.wav


Analyzing file 117 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__187_1308073994223_C.wav
Analyzing file 118 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__189_1308075231945_A.wav
Analyzing file 119 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__189_1308075231945_C.wav
Analyzing file 120 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__191_1308077299430_C.wav
Analyzing file 121 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__191_1308077299430_D.wav
Analyzing file 122 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__193_1308078104592_A.wav
Analyzing file 123 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__193_1308078104592_D.wav
Analyzing file 124 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__194_130813982418

Analyzing file 182 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__282_1311166081161_D.wav
Analyzing file 183 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__284_1311168471850_C.wav
Analyzing file 184 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__286_1311170606028_B.wav
Analyzing file 185 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__287_1311170903290_D.wav
Analyzing file 186 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__288_1311164615284_B.wav
Analyzing file 187 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__290_1311182875320_B.wav
Analyzing file 188 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__293_1311680805936_B.wav
Analyzing file 189 of 195: audio_training_dataset/heartbeat_sounds/Bunlabelledtest__/Bunlabelledtest__293_131168080593

...

Param = 0.00100 - classifier Evaluation Experiment 37 of 100
Param = 0.00100 - classifier Evaluation Experiment 38 of 100
Param = 0.00100 - classifier Evaluation Experiment 39 of 100
Param = 0.00100 - classifier Evaluation Experiment 40 of 100
Param = 0.00100 - classifier Evaluation Experiment 41 of 100
Param = 0.00100 - classifier Evaluation Experiment 42 of 100
Param = 0.00100 - classifier Evaluation Experiment 43 of 100
Param = 0.00100 - classifier Evaluation Experiment 44 of 100
Param = 0.00100 - classifier Evaluation Experiment 45 of 100
Param = 0.00100 - classifier Evaluation Experiment 46 of 100
Param = 0.00100 - classifier Evaluation Experiment 47 of 100
Param = 0.00100 - classifier Evaluation Experiment 48 of 100
Param = 0.00100 - classifier Evaluation Experiment 49 of 100
Param = 0.00100 - classifier Evaluation Experiment 50 of 100
Param = 0.00100 - classifier Evaluation Experiment 51 of 100
Param = 0.00100 - classifier Evaluation Experiment 52 of 100
Param = 0.00100 - classi

Param = 0.01000 - classifier Evaluation Experiment 72 of 100
Param = 0.01000 - classifier Evaluation Experiment 73 of 100
Param = 0.01000 - classifier Evaluation Experiment 74 of 100
Param = 0.01000 - classifier Evaluation Experiment 75 of 100
Param = 0.01000 - classifier Evaluation Experiment 76 of 100
Param = 0.01000 - classifier Evaluation Experiment 77 of 100
Param = 0.01000 - classifier Evaluation Experiment 78 of 100
Param = 0.01000 - classifier Evaluation Experiment 79 of 100
Param = 0.01000 - classifier Evaluation Experiment 80 of 100
Param = 0.01000 - classifier Evaluation Experiment 81 of 100
Param = 0.01000 - classifier Evaluation Experiment 82 of 100
Param = 0.01000 - classifier Evaluation Experiment 83 of 100
Param = 0.01000 - classifier Evaluation Experiment 84 of 100
Param = 0.01000 - classifier Evaluation Experiment 85 of 100
Param = 0.01000 - classifier Evaluation Experiment 86 of 100
Param = 0.01000 - classifier Evaluation Experiment 87 of 100
Param = 0.01000 - classi

Param = 1.00000 - classifier Evaluation Experiment 7 of 100
Param = 1.00000 - classifier Evaluation Experiment 8 of 100
Param = 1.00000 - classifier Evaluation Experiment 9 of 100
Param = 1.00000 - classifier Evaluation Experiment 10 of 100
Param = 1.00000 - classifier Evaluation Experiment 11 of 100
Param = 1.00000 - classifier Evaluation Experiment 12 of 100
Param = 1.00000 - classifier Evaluation Experiment 13 of 100
Param = 1.00000 - classifier Evaluation Experiment 14 of 100
Param = 1.00000 - classifier Evaluation Experiment 15 of 100
Param = 1.00000 - classifier Evaluation Experiment 16 of 100
Param = 1.00000 - classifier Evaluation Experiment 17 of 100
Param = 1.00000 - classifier Evaluation Experiment 18 of 100
Param = 1.00000 - classifier Evaluation Experiment 19 of 100
Param = 1.00000 - classifier Evaluation Experiment 20 of 100
Param = 1.00000 - classifier Evaluation Experiment 21 of 100
Param = 1.00000 - classifier Evaluation Experiment 22 of 100
Param = 1.00000 - classifie

Param = 5.00000 - classifier Evaluation Experiment 42 of 100
Param = 5.00000 - classifier Evaluation Experiment 43 of 100
Param = 5.00000 - classifier Evaluation Experiment 44 of 100
Param = 5.00000 - classifier Evaluation Experiment 45 of 100
Param = 5.00000 - classifier Evaluation Experiment 46 of 100
Param = 5.00000 - classifier Evaluation Experiment 47 of 100
Param = 5.00000 - classifier Evaluation Experiment 48 of 100
Param = 5.00000 - classifier Evaluation Experiment 49 of 100
Param = 5.00000 - classifier Evaluation Experiment 50 of 100
Param = 5.00000 - classifier Evaluation Experiment 51 of 100
Param = 5.00000 - classifier Evaluation Experiment 52 of 100
Param = 5.00000 - classifier Evaluation Experiment 53 of 100
Param = 5.00000 - classifier Evaluation Experiment 54 of 100
Param = 5.00000 - classifier Evaluation Experiment 55 of 100
Param = 5.00000 - classifier Evaluation Experiment 56 of 100
Param = 5.00000 - classifier Evaluation Experiment 57 of 100
Param = 5.00000 - classi

Param = 10.00000 - classifier Evaluation Experiment 76 of 100
Param = 10.00000 - classifier Evaluation Experiment 77 of 100
Param = 10.00000 - classifier Evaluation Experiment 78 of 100
Param = 10.00000 - classifier Evaluation Experiment 79 of 100
Param = 10.00000 - classifier Evaluation Experiment 80 of 100
Param = 10.00000 - classifier Evaluation Experiment 81 of 100
Param = 10.00000 - classifier Evaluation Experiment 82 of 100
Param = 10.00000 - classifier Evaluation Experiment 83 of 100
Param = 10.00000 - classifier Evaluation Experiment 84 of 100
Param = 10.00000 - classifier Evaluation Experiment 85 of 100
Param = 10.00000 - classifier Evaluation Experiment 86 of 100
Param = 10.00000 - classifier Evaluation Experiment 87 of 100
Param = 10.00000 - classifier Evaluation Experiment 88 of 100
Param = 10.00000 - classifier Evaluation Experiment 89 of 100
Param = 10.00000 - classifier Evaluation Experiment 90 of 100
Param = 10.00000 - classifier Evaluation Experiment 91 of 100
Param = 

And that's it, we now have a trained AI model!

## Classification

### Single File
Now to test it out we can use the `file_classification()` function which takes three parameters:
1. **path**: Path to a WAV file to be classified
2. **model_name**: Name of the model we want to use
3. **classifier_type**: "svm" or "knn" or "randomforest" or "gradientboosting" or "extratrees"

We will call the function twice with a file that is a heartbeat sound and one that isn't

In [13]:
aT.file_classification("./audio_training_dataset/test_audio/alarm_beep.wav", "heartbeats", "svm")

(0.0,
 array([0.42218919, 0.45055505, 0.00246762, 0.01666966, 0.00368089,
        0.05327668, 0.00115451, 0.04640905, 0.00359735]),
 ['artifact__',
  'Aunlabelledtest__',
  'Bunlabelledtest__',
  'extrahls__',
  'extrastole__',
  'murmur__',
  'murmur_noisymurmur_',
  'normal__',
  'normal_noisynormal_'])

In [15]:
aT.file_classification("./audio_training_dataset/test_audio/heartbeat.wav", "heartbeats", "svm")

(8.0,
 array([0.00308756, 0.00364194, 0.3846767 , 0.00178296, 0.06651051,
        0.04398587, 0.02003635, 0.0984332 , 0.37784491]),
 ['artifact__',
  'Aunlabelledtest__',
  'Bunlabelledtest__',
  'extrahls__',
  'extrastole__',
  'murmur__',
  'murmur_noisymurmur_',
  'normal__',
  'normal_noisynormal_'])

Now let's examine what this function returned:  
1. First, we have the class_id, which represents the index of the classes array (third field) that the file is most similar to
2. Second, we have a list of probabilities that represent the likelihood of the provided file being of the class in the corresponding index in the classes array
3. Finally, the array with the classes the trained model is able to distinguish

### Multiple Files
pyAudioAnalysis also has a wrapper function that is able to classify multiple files in a directory: 

In [19]:
output = os.popen("python3 ../../../pyAudioAnalysis/pyAudioAnalysis/audioAnalysis.py classifyFolder -i audio_training_dataset/test_audio/ --model svm --classifier heartbeats --details").read()

In [20]:
print(output)

audio_training_dataset/test_audio/alarm_beep.wav	artifact__
audio_training_dataset/test_audio/car_skid.wav	artifact__
audio_training_dataset/test_audio/heartbeat.wav	normal_noisynormal_
audio_training_dataset/test_audio/heartbeat2.wav	Bunlabelledtest__
audio_training_dataset/test_audio/traffic_city.wav	artifact__
artifact__          		3
Aunlabelledtest__   		0
Bunlabelledtest__   		1
extrahls__          		0
extrastole__        		0
murmur__            		0
murmur_noisymurmur_ 		0
normal__            		0
normal_noisynormal_ 		1



As you can see, the heartbeat sounds are correctly identified, but the model also makes the mistake of attributing a heartbeat class to the other files. This problem can be solved by having a more complex model with many more different audio classes. 